In [ ]:
import json 
from pat_sql import PatJob
from pat_common import AppSettings

# Submit Job

In [ ]:
job_para = json.load(open('data/pat_job_test.json'))
job_para

# Run Sequential Code 

In [ ]:
%%time

job = PatJob(job_para) 
if job.job_id> 0:
    job.perform_analysis()    

ret = PatJob.get_results(job.job_id)
res = None
if ret is not None:
    res = pd.DataFrame(json.loads(ret))


# Run with Service

In [22]:
import requests
import uuid
import json

# files = {'para': ('pat_para', open('data/pat_job_test.json','rt'))}
job_para = json.load(open('data/pat_job_test.json'))
job_para['job_guid'] = str(uuid.uuid4())

files = {
        'para': json.dumps(job_para),
        'data': open(r'C:\_Working\PAT_20201019\debug\pat.valid.zip','rb')
    }

ret = requests.post("http://frkcat-ws10:8002/api/Jobs", files=files)

# ret = requests.post("http://localhost:5000/api/Jobs", json=job_para)
ret.status_code, ret.content

(200, b'Analysis submitted: 100000173')

# Results Summary

In [2]:
import requests
import pandas as pd

job_id = 100000171
actions = {
    'list': 'api/Jobs',
    'para': f'api/Jobs/{job_id}/Para',
    'summary': f'api/Jobs/{job_id}',
    'result': f'api/Jobs/{job_id}/Result',
    'validation': f'api/Jobs/{job_id}/Validation',
}

ret = requests.get(f"http://frkcat-ws10:8002/{actions['list']}")
try:
    df = pd.DataFrame(ret.json())
except:
    df = None
    print(ret.content)

df
# print(json.dumps(ret.json(), indent=4))

,job_id,job_guid,job_name,receive_time,update_time,status,user_name,user_email
0,100000173,766e38aa-cccb-4392-a0cb-612994d32a91 ...,PAT_Test_AIG,2021-10-04T06:24:16.906015,2021-10-04T06:25:09.814641,finished,cxiao,chengyou.xiao@guycarp.com
1,100000172,1169970f-caa7-4037-a8e4-5def94ee85ba ...,PAT_Test_AIG,2021-10-04T06:14:12.330143,2021-10-04T06:15:27.186835,finished,cxiao,chengyou.xiao@guycarp.com
2,100000171,80d9e3d2-2625-499e-8d8e-f2aadec9affd ...,PAT_Test_AIG,2021-10-04T06:13:52.048106,2021-10-04T06:14:47.205136,finished,cxiao,chengyou.xiao@guycarp.com


In [ ]:
df.groupby(by='LocationIDStack').agg({'Premium':'sum'}) 